In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import * 

In [0]:
df = spark.read.format("delta")\
    .load("/Volumes/workspace/bronze/bronzevolume/bookings/data")
display(df)


In [0]:
df = df.withColumn("amount", col("amount").cast(DoubleType()))\
    .withColumn("modified_date", current_timestamp())\
    .withColumn("booking_date", to_date(col("booking_date")))\
    .drop("_rescued_data")
display(df)

In [0]:
import dlt
from pyspark.sql.functions import *
from pyspark.sql.types import * 

In [0]:
@dlt.table(
    name= "stage_bookings"
)
def stage_bookings():
    df = spark.readstreamformat("delta")\
        .load("/Volumes/workspace/bronze/bronzevolume/bookings/data")

    return df


In [0]:
@dlt.view(
    name= "trans_bookings"
)
def trans_bookings():
    df = spark.readstream.table("stage_bookings")
    df =  df.withColumn("amount", col("amount").cast(DoubleType()))\
        .withColumn("modified_date", current_timestamp())\
        .withColumn("booking_date", to_date(col("booking_date")))\
        .drop("_rescued_data")
    return df                        


In [0]:
rules = {
    "rule_1" : "booking_id IS NOT NULL",
    "rule_2" : "passenger_id IS NOT NULL"
}

In [0]:
@dlt.table(
    name= "silver_bookings"
)
@dlt.expect_all(rules)
def silver_bookings():
    df = spark.readstream.table("trans_bookings")
    return df

In [0]:
df = spark.read.format("delta")\
    .load("/Volumes/workspace/bronze/bronzevolume/flights/data")

display(df)
